In [69]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pyperclip
from tqdm import tqdm
import numpy as np
from datetime import datetime
import re  
import math
from selenium.webdriver.support.ui import WebDriverWait

In [138]:

driver=webdriver.Chrome()
url='https://www.oliveyoung.co.kr/store/main/getBestList.do' #상품 랭킹 페이지
driver.get(url)


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
items=soup.select('.cate_prd_list')

In [41]:
items[0].select('.prd_info')[0].select('.prd_thumb')[0]['href']

'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000211001&dispCatNo=90000010009&trackingCd=Best_Sellingbest&t_page=랭킹&t_click=판매랭킹_전체_상품상세&t_number=1'

In [43]:
product_info=[]
for i in range(len(items)):
    tmp_list=items[i].select('.prd_info')
    for j in range(len(tmp_list)):
        product_name=items[i].select('.prd_info')[j].select('.tx_name')[0].text
        product_url=items[i].select('.prd_info')[j].select('.prd_thumb')[0]['href']
        product_info.append([product_name, product_url])

In [44]:
product_df=pd.DataFrame(product_info, columns=['product_name', 'product_url'])
product_df['date']=datetime.today().strftime("%Y.%m.%d")

In [45]:
product_df

,product_name,product_url,date
0,[10월 올영픽][JIN 미니포토북 선착순 증정] 라네즈 크림스킨 170ml 기획 ...,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
1,[10월올영픽]에스트라 리제덤 365 모공탄력 캡슐세럼 30ml 기획 (+세럼15m...,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
2,[10월 올영픽] 아누아 복숭아 70 나이아신아마이드 세럼 30ml 더블기획,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
3,[10월올영픽/5번팩증정] 넘버즈인 5번 글루타치온씨 흔적 앰플 30ml 더블 기획...,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
4,[10월 올영픽/Lee콜라보] 이니스프리 레티놀 시카 흔적 앰플 30ml 기획,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
...,...,...,...
95,브링그린 알로에99% 수딩젤 300ml+300ml 더블기획,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
96,라로슈포제 시카플라스트 밤 B5+ 100ml 기획 (+3ml 추가증정),https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
97,[우민누 콜라보/10월 올영픽] 스카이보틀 퍼퓸 핸드크림 50ml 6종 택1,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02
98,[10월 올영픽] 네이밍 플러피 파우더 블러쉬 11colors,https://www.oliveyoung.co.kr/store/goods/getGo...,2024.10.02


In [46]:

def real_crawling():
    tmp_list=[]
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    for a in range(len(soup.select('.inner_list>li'))):
        user_name=soup.select('.inner_list>li')[a].select('.info_user>.id')[0].text
        user_key=soup.select('.inner_list>li')[a].select('.user.clrfix>a')[0]['onclick'].split("'")[1]
        user_url='https://www.oliveyoung.co.kr/store/mypage/getReviewerProfile.do?key=' + user_key
#        review_point=soup.select('.inner_list>li')[a].select('.point')[0].text.split(' ')[-1]
 #       review_date=soup.select('.inner_list>li')[a].select('.date')[0].text
  #      review_text=soup.select('.inner_list>li')[a].select('.txt_inner')[0].text
   #     buy_type = soup.select('.inner_list>li')[a].select('.ico_gift')[0].text
    #    review_referral=soup.select('.inner_list>li')[a].select('.num')[0].text.strip() 
        tmp_list.append([user_name, user_url])
    return tmp_list

In [28]:
def crawling(product_df):
    product_review_df=pd.DataFrame()
    driver.find_element(By.ID, 'searchType_3'.click())
    time.sleep(1)
    
    for d in tqdm(range(len(product_df))):
        product_name=product_df.iloc[d]['product_name']
        url=product_df.iloc[d]['product_url']
        driver.get(url)
        time.sleep(1)

        # 리뷰 버튼 클릭
        driver.find_element(By.ID, value="reviewInfo").click()
        time.sleep(1)

        driver.find_element(By.ID, 'searchType_3'.click())
        time.sleep(1)
        
        # 리뷰 수 저장
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_cnt=int(soup.select('.goods_reputation')[0].text.split('(')[1].split(')')[0].replace(',', ''))
        
        # 상품별 리뷰 추출
        review_list=[]
        if review_cnt==0:
            pass
        
        elif review_cnt>=1000:
            for j in range(0, 9):
                if j == 0: # 우선 첫페이지 (1)에 대해 크롤링 (1~10페이지)
                    # 2페이지부터 다음페이지까지 클릭하면서 각 페이지크롤링
                    for i in range(1, 11):
                        tmp_list=real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            try: # 제품마다 xpath값에 차이가 있어서 구분함
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[7]/'+'a['+str(i)+']')
                            except:
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[8]/'+'a['+str(i)+']')
                            page_box.click()
                        except:
                            pass
                        
                        
                elif j !=  9: #첫 페이지, 마지막 페이지가 아닌 경우 (페이지 11 이후)
                    for i in range(2, 12):
                        tmp_list=real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            try:
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[7]/'+'a['+str(i)+']')
                            except:
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[8]/'+'a['+str(i)+']')
                            page_box.click()
                        except:
                            pass
                
                else: # 마지막페이지의 경우 (넘어가기 인덱스 삭제)
                    for i in range(2, 11):
                        tmp_list=real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            if i!=10:
                                try: # 제품마다 xpath값에 차이가 있어서 구분함
                                    page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[7]/'+'a['+str(i)+']')
                                except:
                                    page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[8]/'+'a['+str(i)+']')
                                page_box.click()      
                        except:
                            pass
                       
        else:
            page_limit=math.ceil(review_cnt/10)
            next_page_cnt=page_limit//10
            for j in range(0, next_page_cnt+1):
                if j == 0: # 우선 첫페이지 (1)에 대해 크롤링
                    # 2페이지부터 다음페이지까지 클릭하면서 각 페이지크롤링
                    for i in range(1, 11):
                        tmp_list=real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            try: # 제품마다 xpath값에 차이가 있어서 구분함
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[7]/'+'a['+str(i)+']')
                            except:
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[8]/'+'a['+str(i)+']')
                            page_box.click()
                        except:
                            pass
                        
                        
                elif j !=  next_page_cnt: #첫 페이지, 마지막 페이지가 아닌 경우 (페이지 11 이후)
                    for i in range(2, 12):
                        tmp_list=real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            try:
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[7]/'+'a['+str(i)+']')
                            except:
                                page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[8]/'+'a['+str(i)+']')
                            page_box.click()
                        except:
                            pass
                
                else: # 마지막페이지의 경우 (넘어가기 인덱스 삭제)
                    for i in range(2, page_limit%10+2):
                        tmp_list=real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            if i!=page_limit%10+1:
                                try: # 제품마다 xpath값에 차이가 있어서 구분함
                                    page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[7]/'+'a['+str(i)+']')
                                except:
                                    page_box = driver.find_element(By.XPATH,'//*[@id="gdasContentsArea"]/div/div[8]/'+'a['+str(i)+']')
                                page_box.click() 
                        except:
                            pass

        review_user_df=pd.DataFrame(review_list, columns=['user_name','user_url'])
        
        product_review_df=pd.concat([product_review_df, review_user_df])
    product_review_df.reset_index(inplace=True, drop=True) # 인덱스 초기화    
    
    return product_review_df

In [75]:
def crawling(product_df):
    product_review_df = pd.DataFrame()  # Initialize the DataFrame to store all reviews

    for d in tqdm(range(len(product_df))):
        product_name = product_df.iloc[d]['product_name']
        url = product_df.iloc[d]['product_url']
        driver.get(url)
        time.sleep(1.5)


        # 리뷰 버튼 클릭
        try:
            driver.find_element(By.ID, value="reviewInfo").click()
            time.sleep(1)
        except Exception as e:
            print(f"{product_name} 상품의 리뷰 버튼 클릭 오류: {e}")
            continue

        # 체크박스 해제 (필요시 클릭)
        try:
            checkbox = driver.find_element(By.ID, 'searchType_3')
            if checkbox.is_selected():  # 체크되어 있으면 해제
                checkbox.click()
            time.sleep(1)
        except Exception as e:
            print(f"{product_name} 상품의 체크박스 해제 오류: {e}")
            continue


        # Save the review count
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_cnt = int(soup.select('.goods_reputation')[0].text.split('(')[1].split(')')[0].replace(',', ''))
        
        # List to hold reviews for the current product
        review_list = []

        # If there are no reviews, continue to the next product
        if review_cnt == 0:
            continue
        
        # If there are a lot of reviews
        elif review_cnt >= 1000:
            for j in range(0, 9):  # Iterate through the pages
                if j == 0:  # For the first page
                    for i in range(1, 11):  # Pages 1 to 10
                        tmp_list = real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            # Attempt to click on the pagination buttons
                            page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{i}]')
                            page_box.click()
                        except:
                            # Try the alternate pagination if the first fails
                            try:
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{i}]')
                                page_box.click()
                            except Exception as e:
                                print(f"Error clicking on page {i} for product {product_name}: {e}")
                                break

                elif j != 9:  # For pages 2 to 10
                    for i in range(2, 12):  # Next pages
                        tmp_list = real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{i}]')
                            page_box.click()
                        except:
                            try:
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{i}]')
                                page_box.click()
                            except Exception as e:
                                print(f"Error clicking on page {i} for product {product_name}: {e}")
                                break
                
                else:  # For the last page
                    for i in range(2, 11):
                        tmp_list = real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            if i != 10:  # Skip clicking on the last page link
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{i}]')
                                page_box.click()
                        except:
                            try:
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{i}]')
                                page_box.click()
                            except Exception as e:
                                print(f"Error clicking on page {i} for product {product_name}: {e}")
                                break

        else:  # If the number of reviews is less than 1000
            page_limit = math.ceil(review_cnt / 10)
            next_page_cnt = page_limit // 10
            
            for j in range(0, next_page_cnt + 1):
                if j == 0:  # For the first page
                    for i in range(1, 11):
                        tmp_list = real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{i}]')
                            page_box.click()
                        except:
                            try:
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{i}]')
                                page_box.click()
                            except Exception as e:
                                print(f"Error clicking on page {i} for product {product_name}: {e}")
                                break

                elif j != next_page_cnt:  # Not the first or last page
                    for i in range(2, 12):
                        tmp_list = real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{i}]')
                            page_box.click()
                        except:
                            try:
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{i}]')
                                page_box.click()
                            except Exception as e:
                                print(f"Error clicking on page {i} for product {product_name}: {e}")
                                break

                else:  # Last page handling
                    for i in range(2, page_limit % 10 + 2):
                        tmp_list = real_crawling()
                        review_list.extend(tmp_list)
                        time.sleep(1)
                        try:
                            if i != page_limit % 10 + 1:
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{i}]')
                                page_box.click()
                        except:
                            try:
                                page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{i}]')
                                page_box.click()
                            except Exception as e:
                                print(f"Error clicking on page {i} for product {product_name}: {e}")
                                break

        # Create DataFrame for the current product's reviews
        review_user_df = pd.DataFrame(review_list, columns=['user_name', 'user_url'])
        
        # Concatenate the current product reviews to the overall DataFrame
        product_review_df = pd.concat([product_review_df, review_user_df])

    product_review_df.reset_index(inplace=True, drop=True)  # Reset the index of the final DataFrame    
    
    return product_review_df

In [80]:
def crawling(product_df):
    product_review_df = pd.DataFrame()  # Initialize the DataFrame to store all reviews
    for d in tqdm(range(len(product_df))):
        product_name = product_df.iloc[d]['product_name']
        url = product_df.iloc[d]['product_url']
        driver.get(url)
        time.sleep(1.5)
        # 리뷰 버튼 클릭
        try:
            driver.find_element(By.ID, value="reviewInfo").click()
            time.sleep(1)
        except Exception as e:
            print(f"{product_name} 상품의 리뷰 버튼 클릭 오류: {e}")
            continue
        # 체크박스 해제 (필요시 클릭)
        try:
            checkbox = driver.find_element(By.ID, 'searchType_3')
            if checkbox.is_selected():  # 체크되어 있으면 해제
                checkbox.click()
            time.sleep(1)
        except Exception as e:
            print(f"{product_name} 상품의 체크박스 해제 오류: {e}")
            continue
        # Save the review count
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        review_cnt = int(soup.select('.goods_reputation')[0].text.split('(')[1].split(')')[0].replace(',', ''))
        # List to hold reviews for the current product
        review_list = []
        # If there are no reviews, continue to the next product
        if review_cnt == 0:
            continue
        # Iterate through pages and collect reviews
        total_reviews = 0
        page_limit = math.ceil(review_cnt / 10) if review_cnt < 1000 else 10  # Limit to 10 pages for large counts
        for j in range(page_limit):  # 페이지 순회
            for i in range(1, 11):  # 각 페이지의 리뷰 크롤링
                tmp_list = real_crawling()
                review_list.extend(tmp_list)
                total_reviews += len(tmp_list)
                # 1000개 이상의 리뷰가 모이면 다음 상품으로 넘어감
                if total_reviews >= 1000:
                    print(f"{product_name}에서 1000개 리뷰를 모았습니다. 다음 상품으로 넘어갑니다.")
                    break
                time.sleep(1)
                try:
                    # 페이지 넘기기 시도
                    page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[7]/a[{i}]')
                    page_box.click()
                except:
                    try:
                        page_box = driver.find_element(By.XPATH, f'//*[@id="gdasContentsArea"]/div/div[8]/a[{i}]')
                        page_box.click()
                    except Exception as e:
                        print(f"Error clicking on page {i} for product {product_name}: {e}")
                        break
            if total_reviews >= 1000:
                break
        # Create DataFrame for the current product's reviews
        review_user_df = pd.DataFrame(review_list, columns=['user_name', 'user_url'])
        # Concatenate the current product reviews to the overall DataFrame
        product_review_df = pd.concat([product_review_df, review_user_df])
        # 다음 상품으로 넘어감
        if total_reviews >= 1000:
            continue
    product_review_df.reset_index(inplace=True, drop=True)  # Reset the index of the final DataFrame
    return product_review_df

In [53]:

df1=product_df[:10]
df2=product_df[10:20]
df3=product_df[20:30]
df4=product_df[30:40]
df5=product_df[40:50]
df6=product_df[50:60]
df7=product_df[60:70]
df8=product_df[70:80]
df9=product_df[80:90]
df10=product_df[90:]

In [84]:
# driver=webdriver.Chrome()
# product_review_df1=crawling(df1)

# product_review_df1.to_csv('user_data/product_review_df1.csv', index=False)

# product_review_df2=crawling(df2)
# product_review_df2.to_csv('user_data/product_review_df2.csv', index=False)

# product_review_df3=crawling(df3)
# product_review_df3.to_csv('user_data/product_review_df3.csv', index=False)

# product_review_df4=crawling(df4)
# product_review_df4.to_csv('user_data/product_review_df4.csv', index=False)

# product_review_df5=crawling(df5)
# product_review_df5.to_csv('user_data/product_review_df5.csv', index=False)

# product_review_df6=crawling(df6)
# product_review_df6.to_csv('user_data/product_review_df6.csv', index=False)

product_review_df7=crawling(df7)
product_review_df7.to_csv('user_data/product_review_df7.csv', index=False)

product_review_df8=crawling(df8)
product_review_df8.to_csv('user_data/product_review_df8.csv', index=False)

product_review_df9=crawling(df9)
product_review_df9.to_csv('user_data/product_review_df9.csv', index=False)

# product_review_df10=crawling(df10)
# product_review_df10.to_csv('user_data/product_review_df10.csv', index=False)

 10%|█         | 1/10 [02:35<23:22, 155.81s/it]

Error clicking on page 1 for product [NEW기획출시] 투크 워터프루프 슬림 아이라이너 15colors (기획/단품): Message: element not interactable
  (Session info: chrome=129.0.6668.71)
Stacktrace:
0   chromedriver                        0x00000001032e0248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001032d8730 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000102eec0d0 cxxbridge1$string$len + 89088
3   chromedriver                        0x0000000102f314cc cxxbridge1$string$len + 372732
4   chromedriver                        0x0000000102f27034 cxxbridge1$string$len + 330596
5   chromedriver                        0x0000000102f26ad0 cxxbridge1$string$len + 329216
6   chromedriver                        0x0000000102f6a7d0 cxxbridge1$string$len + 606976
7   chromedriver                        0x0000000102f2512c cxxbridge1$string$len + 322652
8   chromedriver                        0x0000000102f25d7c cxxbridge1$string$len + 325804
9   chromedriver           

 20%|██        | 2/10 [03:14<11:36, 87.05s/it] 

Error clicking on page 2 for product [NEW컬러] 투쿨포스쿨 프로타주 펜슬 샤프너 증정기획(9,11,14,15호 한정): Message: element not interactable
  (Session info: chrome=129.0.6668.71)
Stacktrace:
0   chromedriver                        0x00000001032e0248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001032d8730 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000102eec0d0 cxxbridge1$string$len + 89088
3   chromedriver                        0x0000000102f314cc cxxbridge1$string$len + 372732
4   chromedriver                        0x0000000102f27034 cxxbridge1$string$len + 330596
5   chromedriver                        0x0000000102f26ad0 cxxbridge1$string$len + 329216
6   chromedriver                        0x0000000102f6a7d0 cxxbridge1$string$len + 606976
7   chromedriver                        0x0000000102f2512c cxxbridge1$string$len + 322652
8   chromedriver                        0x0000000102f25d7c cxxbridge1$string$len + 325804
9   chromedriver         

 30%|███       | 3/10 [03:24<06:03, 51.95s/it]

...

[립1위]롬앤 쥬시 래스팅 틴트 상품의 체크박스 해제 오류: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchType_3"]"}
  (Session info: chrome=129.0.6668.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001032e0248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001032d8730 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000102eec260 cxxbridge1$string$len + 89488
3   chromedriver                        0x0000000102f3050c cxxbridge1$string$len + 368700
4   chromedriver                        0x0000000102f6a7d0 cxxbridge1$string$len + 606976
5   chromedriver                        0x0000000102f2512c cxxbridge1$string$len + 322652
6   chromedriver                        0x0000000102f25d7c cxxbridge1$string$len + 325804
7   chromedriver                 

 90%|█████████ | 9/10 [20:39<02:01, 121.85s/it]

[NEW 출시] 투크 윗아웃 미러 립 틴트 13종 중 택1에서 1000개 리뷰를 모았습니다. 다음 상품으로 넘어갑니다.
Error clicking on page 3 for product [변우석 PICK] 크리니크 치크 팝 단품/기획 택1: Message: element not interactable
  (Session info: chrome=129.0.6668.71)
Stacktrace:
0   chromedriver                        0x00000001032e0248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001032d8730 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000102eec0d0 cxxbridge1$string$len + 89088
3   chromedriver                        0x0000000102f314cc cxxbridge1$string$len + 372732
4   chromedriver                        0x0000000102f27034 cxxbridge1$string$len + 330596
5   chromedriver                        0x0000000102f26ad0 cxxbridge1$string$len + 329216
6   chromedriver                        0x0000000102f6a7d0 cxxbridge1$string$len + 606976
7   chromedriver                        0x0000000102f2512c cxxbridge1$string$len + 322652
8   chromedriver                        0x0000000102f25d7c cxxbri

100%|██████████| 10/10 [21:30<00:00, 129.09s/it]

Error clicking on page 3 for product [변우석 PICK] 크리니크 치크 팝 단품/기획 택1: Message: element not interactable
  (Session info: chrome=129.0.6668.71)
Stacktrace:
0   chromedriver                        0x00000001032e0248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x00000001032d8730 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x0000000102eec0d0 cxxbridge1$string$len + 89088
3   chromedriver                        0x0000000102f314cc cxxbridge1$string$len + 372732
4   chromedriver                        0x0000000102f27034 cxxbridge1$string$len + 330596
5   chromedriver                        0x0000000102f26ad0 cxxbridge1$string$len + 329216
6   chromedriver                        0x0000000102f6a7d0 cxxbridge1$string$len + 606976
7   chromedriver                        0x0000000102f2512c cxxbridge1$string$len + 322652
8   chromedriver                        0x0000000102f25d7c cxxbridge1$string$len + 325804
9   chromedriver                        0x

In [79]:
# 중복된 행 확인
duplicate_rows = product_review_df1[product_review_df1.duplicated()]

# 중복된 행 출력
print(f"중복된 행 수: {len(duplicate_rows)}")

중복된 행 수: 2000


실제 크롤링

In [85]:
from glob import glob

In [116]:
df_list = sorted(glob("user_data/*.csv"))

In [117]:
total=pd.DataFrame()
for i in range(0, len(df_list)):
    tmp=pd.read_csv(df_list[i])
    total=pd.concat([total, tmp])

In [120]:
# 중복된 행 확인
duplicate_rows = total[total.duplicated()]

# 중복된 행 출력
print(f"중복된 행 수: {len(duplicate_rows)}")
print(duplicate_rows)

중복된 행 수: 69597
      user_name                                           user_url
4           yop  https://www.oliveyoung.co.kr/store/mypage/getR...
10         려워니진  https://www.oliveyoung.co.kr/store/mypage/getR...
11    rose7****  https://www.oliveyoung.co.kr/store/mypage/getR...
12           별향  https://www.oliveyoung.co.kr/store/mypage/getR...
13          yop  https://www.oliveyoung.co.kr/store/mypage/getR...
...         ...                                                ...
8265       치코리따  https://www.oliveyoung.co.kr/store/mypage/getR...
8266       김치보끔  https://www.oliveyoung.co.kr/store/mypage/getR...
8267     딸기우윳빛깔  https://www.oliveyoung.co.kr/store/mypage/getR...
8268      충분히예뻐  https://www.oliveyoung.co.kr/store/mypage/getR...
8269       자몽허니  https://www.oliveyoung.co.kr/store/mypage/getR...

[69597 rows x 2 columns]


In [123]:
total.user_name.nunique()

5678

In [124]:
total.shape[0], total.user_url.nunique()

(75323, 5725)

In [125]:

total=total.groupby(['user_name','user_url']).count()
total.reset_index(inplace=True)

In [126]:
total.shape[0]

5726

In [127]:
total.to_csv("total.csv")

In [128]:
total=pd.read_csv('total.csv')
user_list=list(total['user_url'])

In [173]:
def user_review_crawling(user_list):
    review_data=pd.DataFrame()
    for i in tqdm(user_list):
        url = i
        driver.get(url)
        
        # 프로필 비공개한 유저 pass
        html = driver.page_source
        soup3 = BeautifulSoup(html, 'html.parser')
        rv_num = soup3.find('div', {"class":"reviewer-profile-content__header"})   
        if str(type(rv_num))=="<class 'bs4.element.Tag'>" and rv_num.select('div>span')[0].text.strip()!='-':
            last_height = driver.execute_script("return document.body.scrollHeight")
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(1)  # 로딩을 기다립니다.
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
            time.sleep(1) 
            html = driver.page_source
            soup3 = BeautifulSoup(html, 'html.parser')

            # 리뷰어 이름
            rv_name = soup3.select('.id.my-profile.on>strong')   # [Ctrl + Shift + I]로 코드 확인 가능
            for info in rv_name:
              rv_name=info.get_text().strip()

            # 리뷰어 랭킹(없음)
            rv_rank = soup3.find_all('span', {"class":"badge-item top-number"})   # [Ctrl + Shift + I]로 코드 확인 가능
            for info in rv_rank:
              rv_rank=info.get_text().strip()

            # 리뷰어 피부타입
            rv_type = soup3.find_all('li', {"class":"list-item"})   # [Ctrl + Shift + I]로 코드 확인 가능
            type_list=[]
            for info in rv_type:
              type_list.append(info.get_text().strip())

            # 리뷰어 도움/팔로워/팔로잉 
            help_num=int(soup3.select('.reviewer-profile-info__bottom.reviewer-data-info>.reviewer-data-info__list')[0].select('.num')[0].text.strip().replace(',',''))
            follower_num=int(soup3.select('.reviewer-profile-info__bottom.reviewer-data-info>.reviewer-data-info__list')[1].select('.num')[0].text.strip().replace(',',''))
            following_num=int(soup3.select('.reviewer-profile-info__bottom.reviewer-data-info>.reviewer-data-info__list')[2].select('.num')[0].text.strip().replace(',',''))

            # 리뷰어 누적 리뷰수 → 문제 있음(필요없는 부분 있음)
            rv_num = soup3.find('div', {"class":"reviewer-profile-content__header"})   # [Ctrl + Shift + I]로 코드 확인 가능
            rv_num=rv_num.select('div>span')[0].text.strip()

            reviews=soup3.select('.rw-box')
            review_info=[]
            for review in reviews:
                brand=review.select('.rw-box-figcaption__brand') # 브랜드
                for info in brand:
                    brand=info.text.strip()

                product_name=review.select('.rw-box-figcaption__name') #제품 이름
                for info in product_name:
                    product_name=info.text

                product_option=review.select('.rw-box-figcaption__sub') # 제품 옵션
                for info in product_option:
                  product_option=info.text.strip()
                 
                review_point=review.select('.point')[0].text.strip() #리뷰 평점
                review_date=review.select('.review_point_text')[0].text.strip() #리뷰 작성일 
                review_text=review.select('.rw-box__description')[0].text.strip() #리뷰 내용
                review_referral=int(review.select('.rw-box__help .num')[0].text.strip()) #리뷰 추천수
                
                # 재구매 여부 확인 (요소가 존재하는지 확인)
                if review.select('.rx-box__second-line'):
                    repurchase = review.select('.rx-box__second-line')[0].text.strip()
                else:
                    repurchase = None  # 해당 요소가 없는 경우 빈 값 설정
                
                # 구매처 확인 (요소가 존재하는지 확인)
                if review.select('.ic_offlineStore'):
                    purchase_place = review.select('.ic_offlineStore')[0].text.strip()
                else:
                    purchase_place = None  # 해당 요소가 없는 경우 빈 값 설정

                review_info.append([brand,product_name, product_option, review_point, review_date, review_text, review_referral,repurchase,purchase_place])

            review_df=pd.DataFrame(review_info, columns=['brand','product_name','product_option','review_point','review_date','review_text','review_referral','repurchase','purchase_place'])

            review_df['user_name']=rv_name
            review_df['user_rank']=str(rv_rank)
            review_df['user_type']=str(type_list)
            review_df['user_help_num']=help_num
            review_df['user_follower_num']=follower_num
            review_df['user_following_num']=following_num
            
            review_data=pd.concat([review_data, review_df])
    return review_data

In [131]:

user_list1_1=user_list[:500]
user_list1_2=user_list[500:1500]
user_list1_3=user_list[1500:4000]
user_list2=user_list[4000:5000]
# user_list3=user_list[8000:12000]
# user_list4=user_list[12000:16000]
# user_list5=user_list[16000:20000]
# user_list6=user_list[20000:24000]
# user_list7=user_list[24000:28000]
# user_list8=user_list[28000:32000]
# user_list9=user_list[32000:]

In [190]:
user_list_1=user_list[:1000]
user_list_2=user_list[1000:2000]
user_list_3=user_list[2000:3000]
user_list_4=user_list[3000:4000]
user_list_5=user_list[4000:5000]
user_list_6=user_list[5000:]

In [176]:
driver.quit()
driver=webdriver.Chrome()
review_data_1=user_review_crawling(user_list_1)
review_data_1.reset_index(inplace=True, drop=True)
review_data_1.to_csv('user_review_data/review_data1_1.csv', index=False)

100%|██████████| 1000/1000 [5:16:36<00:00, 19.00s/it]  


In [177]:

driver.quit()
driver=webdriver.Chrome()
review_data_2=user_review_crawling(user_list_2)
review_data_2.reset_index(inplace=True, drop=True)
review_data_2.to_csv('user_review_data/review_data1_2.csv', index=False)

100%|██████████| 1000/1000 [4:12:19<00:00, 15.14s/it]  


In [178]:

driver.quit()
driver=webdriver.Chrome()
review_data_3=user_review_crawling(user_list_3)
review_data_3.reset_index(inplace=True, drop=True)
review_data_3.to_csv('user_review_data/review_data1_3.csv', index=False)

100%|██████████| 1000/1000 [6:22:04<00:00, 22.92s/it]  


In [180]:
user_list_4[359]

'https://www.oliveyoung.co.kr/store/mypage/getReviewerProfile.do?key=K1ZBVGxBdEVVVEtTZm5FT3hqZVkxdz09'

In [183]:

driver.quit()
driver=webdriver.Chrome()
del user_list_4[359]
review_data_4=user_review_crawling(user_list_4)
review_data_4.reset_index(inplace=True, drop=True)
review_data_4.to_csv('user_review_data/review_data1_4.csv', index=False)

100%|██████████| 999/999 [7:03:48<00:00, 25.45s/it]    


In [ ]:
user_list_4[359]

In [188]:

driver.quit()
driver=webdriver.Chrome()
del user_list_5[327]
review_data_5=user_review_crawling(user_list_5)
review_data_5.reset_index(inplace=True, drop=True)
review_data_5.to_csv('user_review_data/review_data1_5.csv', index=False)

100%|██████████| 999/999 [7:15:19<00:00, 26.15s/it]    


In [193]:

driver.quit()
driver=webdriver.Chrome()
del user_list_6[653]
review_data_6=user_review_crawling(user_list_6)
                                   
review_data_6.reset_index(inplace=True, drop=True)
review_data_6.to_csv('user_review_data/review_data1_6.csv', index=False)

100%|██████████| 725/725 [5:23:43<00:00, 26.79s/it]    
